<a href="https://colab.research.google.com/github/sophia-zhang-qwq/animal-pose-est/blob/main/docs/notebooks/sleap_on_2fruitflies_example_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training and inference on an example dataset

In this notebook we'll install SLEAP, download a sample dataset, run training and inference on that dataset using the SLEAP command-line interface, and then download the predictions.

# SLEAP Requirements

---
## SLEAP Software Requirements:
*   All training videos must have the same aspect ratio
  *   *see: [https://github.com/talmolab/sleap/issues/2004](https://github.com/talmolab/sleap/issues/2004)*
*   Input scaling must be set to 1
  *   *see: [https://github.com/talmolab/sleap/discussions/925](https://github.com/talmolab/sleap/discussions/925)*
*   Python version must be <=3.10
  *   *see: [https://github.com/talmolab/sleap/issues/2100](https://github.com/talmolab/sleap/issues/2100)*
  *   *guide:[https://gist.github.com/kargaranamir/e0b7910fed0a31](https://gist.github.com/kargaranamir/e0b7910fed0a3189563d9254c7a2c439)*
*   Must use CUDA 11.2 and cuDNN 8.1
  *   *see: [https://github.com/DeepLabCut/DeepLabCut/issues/2465](https://github.com/DeepLabCut/DeepLabCut/issues/2465)*
---

# Environment Setup

In [2]:
# Configure Training Environment

# override and install python 3.10
! wget -O mini.sh https://repo.anaconda.com/miniconda/Miniconda3-py310_22.11.1-1-Linux-x86_64.sh
! chmod +x mini.sh
! bash ./mini.sh -b -f -p /usr/local
! conda install -q -y jupyter
! conda install -q -y google-colab -c conda-forge
! python -m ipykernel install --name "py310" --user

--2025-04-19 23:08:04--  https://repo.anaconda.com/miniconda/Miniconda3-py310_22.11.1-1-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.191.158, 104.16.32.241, 2606:4700::6810:bf9e, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.191.158|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 72402405 (69M) [application/x-sh]
Saving to: ‘mini.sh’

mini.sh             100%[===================>]  69.05M   127MB/s    in 0.5s    

2025-04-19 23:08:04 (127 MB/s) - ‘mini.sh’ saved [72402405/72402405]

PREFIX=/usr/local
Unpacking payload ...
                                                                               
Installing base environment...





Preparing transaction: - \ | / done
Executing transaction: \ | / - \ | / - \ | / - \ | / - \ | / - \ done
installation finished.
    You currently have a PYTHONPATH environment variable set. This may cause
    unexpected behavior when

In [3]:
# Install SLEAP & Dependencies

# confirm using python 3.10
! python3 --version

Python 3.10.16


In [4]:
# install sleap and dependencies
# pypi := extra packages such as tf for sleap dependency
! pip install sleap[pypi]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 904.1/904.1 kB 74.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 64.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.8/28.8 MB 50.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.3/214.3 kB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 MB 11.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 64.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 107.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 111.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 88.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.6/290.6 kB 42.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
# install additional sleap dependencies and load GPU support through CUDA
! pip install matplotlib-inline
! pip install ipython
! apt-get install cuda-11-8
! apt-get install -y libcudnn8=8.6.0.163-1+cuda11.8
! pip install numpy==1.23

! export PATH=/usr/local/cuda-11.8/bin:$PATH
! export LD_LIBRARY_PATH=/usr/local/cuda-11.8/lib64:$LD_LIBRARY_PATH
! export CUDNN_INCLUDE_DIR=/usr/local/cuda/include
! export CUDNN_LIB_DIR=/usr/local/cuda/lib64

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  cpp-12 cuda-cccl-11-8 cuda-command-line-tools-11-8 cuda-compiler-11-8
  cuda-cudart-11-8 cuda-cudart-dev-11-8 cuda-cuobjdump-11-8 cuda-cupti-11-8
  cuda-cupti-dev-11-8 cuda-cuxxfilt-11-8 cuda-demo-suite-11-8
  cuda-documentation-11-8 cuda-driver-dev-11-8 cuda-drivers cuda-drivers-570
  cuda-gdb-11-8 cuda-libraries-11-8 cuda-libraries-dev-11-8 cuda-memcheck-11-8
  cuda-nsight-11-8 cuda-nsight-compute-11-8 cuda-nsight-systems-11-8
  cuda-nvcc-11-8 cuda-nvdisasm-11-8 cuda-nvml-dev-11-8 cuda-nvprof-11-8
  cuda-nvprune-11-8 cuda-nvrtc-11-8 cuda-nvrtc-dev-11-8 cuda-nvtx-11-8
  cuda-nvvp-11-8 cuda-profiler-api-11-8 cuda-runtime-11-8 cuda-sanitizer-11-8
  cuda-toolkit-11-8 cuda-toolkit-11-8-config-common
  cuda-toolkit-11-config-common cuda-tools-11-8 cuda-visual-tools-11-8
  dctrl-tools default-jre default-jre-headless dkms fakeroot fonts-deja

In [ ]:
# restart runtime so GPU changes take effect
import os
os._exit(0)

## Install SLEAP
Note: Before installing SLEAP check [SLEAP releases](https://github.com/talmolab/sleap/releases) page for the latest version.

In [1]:
!pip show sleap
#import sleap
#print(sleap.__version__)
#ModuleNotFoundError: No module named 'sleap'

Name: sleap
Version: 1.4.1
Summary: SLEAP (Social LEAP Estimates Animal Poses) is a deep learning framework for animal pose tracking.
Home-page: https://sleap.ai
Author: Talmo Pereira
Author-email: talmo@salk.edu
License: BSD 3-Clause License
Location: /usr/local/lib/python3.10/site-packages
Requires: imgstore, nixio, pynwb, qimage2ndarray, segmentation-models
Required-by: 


In [2]:
# This initializes the GPU and prevents TensorFlow from filling the entire GPU memory
! python -c "import sleap; sleap.disable_preallocation(); sleap.versions(); sleap.system_summary()"

INFO:numexpr.utils:NumExpr defaulting to 2 threads.
INFO:matplotlib.font_manager:generated new fontManager
2025-04-19 23:26:15.075896: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-04-19 23:26:16.272798: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-04-19 23:26:16.273256: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
SLEAP: 1.4.1
TensorFlow: 2.8.4
Numpy: 1.23.0
Python: 3.10.16
OS: Linux-6.1.123+-x86_64-with-glibc2.35
GPUs: 1/1 available
  Device: /physical_device:GPU:0
         Available: True
       Initialized: False
     Memory growth: True


In [4]:
!pip install jsmin --quiet
!pip install imgaug --quiet
!pip install pyzmq --quiet
!pip install tensorflow_hub --quiet
!pip install pykalman --quiet
!pip install seaborn --quiet
!pip install cattrs --quiet

## Download sample training data into Colab
Let's download a sample dataset from the SLEAP [sample datasets repository](https://github.com/talmolab/sleap-datasets) into Colab.

In [5]:
!apt-get install tree
!wget -O dataset.zip https://github.com/talmolab/sleap-datasets/releases/download/dm-courtship-v1/drosophila-melanogaster-courtship.zip
!mkdir dataset
!unzip dataset.zip -d dataset
!rm dataset.zip
!tree dataset

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 34 not upgraded.
Need to get 47.9 kB of archives.
After this operation, 116 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tree amd64 2.0.2-1 [47.9 kB]
Fetched 47.9 kB in 1s (67.6 kB/s)
Selecting previously unselected package tree.
(Reading database ... 133187 files and directories currently installed.)
Preparing to unpack .../tree_2.0.2-1_amd64.deb ...
Unpacking tree (2.0.2-1) ...
Setting up tree (2.0.2-1) ...
Processing triggers for man-db (2.10.2-1) ...
--2025-04-19 23:28:48--  https://github.com/talmolab/sleap-datasets/releases/download/dm-courtship-v1/drosophila-melanogaster-courtship.zip
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting

## Train models
For the top-down pipeline, we'll need train two models: a centroid model and a centered-instance model.

Using the command-line interface, we'll first train a model for centroids using the default **training profile**. The training profile determines the model architecture, the learning rate, and other parameters.

When you start training, you'll first see the training parameters and then the training and validation loss for each training epoch.

As soon as you're satisfied with the validation loss you see for an epoch during training, you're welcome to stop training by clicking the stop button. The version of the model with the lowest validation loss is saved during training, and that's what will be used for inference.

If you don't stop training, it will run for 200 epochs or until validation loss fails to improve for some number of epochs (controlled by the `early_stopping` fields in the training profile).

In [6]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
#!pip install tensorflow-gpu==2.11.0

Num GPUs Available:  1


In [7]:
!nvcc --version
!nvidia-smi

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0
Sat Apr 19 23:29:48 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   33C    P8       

In [8]:
#import os
#os.environ['MPLBACKEND'] = 'Agg'  # or 'inline' for Jupyter/Colab
#~40min train time
!sleap-train baseline.centroid.json "dataset/drosophila-melanogaster-courtship/courtship_labels.slp" --run_name "courtship.centroid" --video-paths "dataset/drosophila-melanogaster-courtship/20190128_113421.mp4"

INFO:numexpr.utils:NumExpr defaulting to 2 threads.
INFO:sleap.nn.training:Versions:
SLEAP: 1.4.1
TensorFlow: 2.8.4
Numpy: 1.23.0
Python: 3.10.16
OS: Linux-6.1.123+-x86_64-with-glibc2.35
INFO:sleap.nn.training:Training labels file: dataset/drosophila-melanogaster-courtship/courtship_labels.slp
INFO:sleap.nn.training:Training profile: /usr/local/lib/python3.10/site-packages/sleap/training_profiles/baseline.centroid.json
INFO:sleap.nn.training:
INFO:sleap.nn.training:Arguments:
INFO:sleap.nn.training:{
    "training_job_path": "baseline.centroid.json",
    "labels_path": "dataset/drosophila-melanogaster-courtship/courtship_labels.slp",
    "video_paths": [
        "dataset/drosophila-melanogaster-courtship/20190128_113421.mp4"
    ],
    "val_labels": null,
    "test_labels": null,
    "base_checkpoint": null,
    "tensorboard": false,
    "save_viz": false,
    "keep_viz": false,
    "zmq": false,
    "publish_port": 9001,
    "controller_port": 9000,
    "run_name": "courtship.centroid

Let's now train a centered-instance model.

In [10]:
#~10mins
!sleap-train baseline_medium_rf.topdown.json "dataset/drosophila-melanogaster-courtship/courtship_labels.slp" --run_name "courtship.topdown_confmaps" --video-paths "dataset/drosophila-melanogaster-courtship/20190128_113421.mp4"

INFO:numexpr.utils:NumExpr defaulting to 2 threads.
INFO:sleap.nn.training:Versions:
SLEAP: 1.4.1
TensorFlow: 2.8.4
Numpy: 1.23.0
Python: 3.10.16
OS: Linux-6.1.123+-x86_64-with-glibc2.35
INFO:sleap.nn.training:Training labels file: dataset/drosophila-melanogaster-courtship/courtship_labels.slp
INFO:sleap.nn.training:Training profile: /usr/local/lib/python3.10/site-packages/sleap/training_profiles/baseline_medium_rf.topdown.json
INFO:sleap.nn.training:
INFO:sleap.nn.training:Arguments:
INFO:sleap.nn.training:{
    "training_job_path": "baseline_medium_rf.topdown.json",
    "labels_path": "dataset/drosophila-melanogaster-courtship/courtship_labels.slp",
    "video_paths": [
        "dataset/drosophila-melanogaster-courtship/20190128_113421.mp4"
    ],
    "val_labels": null,
    "test_labels": null,
    "base_checkpoint": null,
    "tensorboard": false,
    "save_viz": false,
    "keep_viz": false,
    "zmq": false,
    "publish_port": 9001,
    "controller_port": 9000,
    "run_name": "

## train a centroid model: baseline.centroid.json

!sleap-train baseline.centroid.json "dataset/drosophila-melanogaster-courtship/courtship_labels.slp" --run_name "courtship.centroid" --video-paths "dataset/drosophila-melanogaster-courtship/20190128_113421.mp4"

## train a centered-instance model: baseline_medium_rf.topdown.json

!sleap-train baseline_medium_rf.topdown.json "dataset/drosophila-melanogaster-courtship/courtship_labels.slp" --run_name "courtship.topdown_confmaps" --video-paths "dataset/drosophila-melanogaster-courtship/20190128_113421.mp4"

The models (along with the profiles and ground truth data used to train and validate the model) are saved in the `models/` directory:

In [12]:
!tree models/

models/
├── courtship.centroid
│   ├── best_model.h5
│   ├── initial_config.json
│   ├── labels_gt.train.slp
│   ├── labels_gt.val.slp
│   ├── labels_pr.train.slp
│   ├── labels_pr.val.slp
│   ├── metrics.train.npz
│   ├── metrics.val.npz
│   ├── training_config.json
│   └── training_log.csv
└── courtship.topdown_confmaps
    ├── best_model.h5
    ├── initial_config.json
    ├── labels_gt.train.slp
    ├── labels_gt.val.slp
    ├── labels_pr.train.slp
    ├── labels_pr.val.slp
    ├── metrics.train.npz
    ├── metrics.val.npz
    ├── training_config.json
    └── training_log.csv

2 directories, 20 files


## Inference
Let's run inference with our trained models for centroids and centered instances.

In [18]:
!sleap-track "dataset/drosophila-melanogaster-courtship/20190128_113421.mp4" --frames 0-100 -m "models/courtship.centroid" -m "models/courtship.topdown_confmaps"

INFO:numexpr.utils:NumExpr defaulting to 2 threads.
Started inference at: 2025-04-20 00:38:24.009820
Args:
{
│   'data_path': 'dataset/drosophila-melanogaster-courtship/20190128_113421.mp4',
│   'models': [
│   │   'models/courtship.centroid',
│   │   'models/courtship.topdown_confmaps'
│   ],
│   'frames': '0-100',
│   'only_labeled_frames': False,
│   'only_suggested_frames': False,
│   'output': None,
│   'no_empty_frames': False,
│   'verbosity': 'rich',
│   'video.dataset': None,
│   'video.input_format': 'channels_last',
│   'video.index': '',
│   'cpu': False,
│   'first_gpu': False,
│   'last_gpu': False,
│   'gpu': 'auto',
│   'max_edge_length_ratio': 0.25,
│   'dist_penalty_weight': 1.0,
│   'batch_size': 4,
│   'open_in_gui': False,
│   'peak_threshold': 0.2,
│   'max_instances': None,
│   'tracking.tracker': None,
│   'tracking.max_tracking': None,
│   'tracking.max_tracks': None,
│   'tracking.target_instance_count': None,
│   'tracking.pre_cull_to_target': None,
│   'trac

When inference is finished, predictions are saved in a file. Since we didn't specify a path, it will be saved as `<video filename>.predictions.slp` in the same directory as the video:

In [21]:
# built-in initial_config.json at sleap github
# https://github.com/talmolab/sleap/tree/main/sleap/training_profiles
# baseline.centroid.json
# baseline_medium_rf.topdown.json
!tree /content

/content
├── dataset
│   ├── drosophila-melanogaster-courtship
│   │   ├── 20190128_113421.mp4
│   │   ├── 20190128_113421.predictions.slp
│   │   ├── courtship_labels.slp
│   │   └── example.jpg
│   └── __MACOSX
│       └── drosophila-melanogaster-courtship
├── mini.sh
├── models
│   ├── courtship.centroid
│   │   ├── best_model.h5
│   │   ├── initial_config.json
│   │   ├── labels_gt.train.slp
│   │   ├── labels_gt.val.slp
│   │   ├── labels_pr.train.slp
│   │   ├── labels_pr.val.slp
│   │   ├── metrics.train.npz
│   │   ├── metrics.val.npz
│   │   ├── training_config.json
│   │   └── training_log.csv
│   └── courtship.topdown_confmaps
│       ├── best_model.h5
│       ├── initial_config.json
│       ├── labels_gt.train.slp
│       ├── labels_gt.val.slp
│       ├── labels_pr.train.slp
│       ├── labels_pr.val.slp
│       ├── metrics.train.npz
│       ├── metrics.val.npz
│       ├── training_config.json
│       └── training_log.csv
├── sample_data
│   ├── anscombe.json
│   ├── califo

In [25]:
!zip -r sleap_project_backup.zip /content
from google.colab import files
files.download("sleap_project_backup.zip")

updating: content/ (stored 0%)
updating: content/.config/ (stored 0%)
updating: content/.config/.last_opt_in_prompt.yaml (stored 0%)
updating: content/.config/gce (stored 0%)
updating: content/.config/active_config (stored 0%)
updating: content/.config/default_configs.db (deflated 98%)
updating: content/.config/.last_survey_prompt.yaml (stored 0%)
updating: content/.config/.last_update_check.json (deflated 23%)
updating: content/.config/hidden_gcloud_config_universe_descriptor_data_cache_configs.db (deflated 97%)
updating: content/.config/config_sentinel (stored 0%)
updating: content/.config/configurations/ (stored 0%)
updating: content/.config/configurations/config_default (deflated 15%)
updating: content/.config/logs/ (stored 0%)
updating: content/.config/logs/2025.04.17/ (stored 0%)
updating: content/.config/logs/2025.04.17/13.36.23.688038.log (deflated 57%)
updating: content/.config/logs/2025.04.17/13.36.05.735198.log (deflated 58%)
updating: content/.config/logs/2025.04.17/13.36.1

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [23]:
!tree dataset/drosophila-melanogaster-courtship

dataset/drosophila-melanogaster-courtship
├── 20190128_113421.mp4
├── 20190128_113421.predictions.slp
├── courtship_labels.slp
└── example.jpg

0 directories, 4 files


You can inspect your predictions file using `sleap-inspect`:

In [24]:
!sleap-inspect dataset/drosophila-melanogaster-courtship/20190128_113421.predictions.slp

INFO:numexpr.utils:NumExpr defaulting to 2 threads.
/usr/local/lib/python3.10/site-packages/networkx/readwrite/json_graph/node_link.py:287: FutureWarning: 
The default value will be changed to `edges="edges" in NetworkX 3.6.

To make this warning go away, explicitly set the edges kwarg, e.g.:

  nx.node_link_graph(data, edges="links") to preserve current behavior, or
  nx.node_link_graph(data, edges="edges") for forward compatibility.
  warnings.warn(
Labeled frames: 101
Tracks: 0
Video files:
  dataset/drosophila-melanogaster-courtship/20190128_113421.mp4
    labeled frames: 101
    labeled frames from 0 to 100
    user labeled frames: 0
    tracks: 1
    max instances in frame: 2
Total user labeled frames: 0

Provenance:
  model_paths: ['models/courtship.centroid/training_config.json', 'models/courtship.topdown_confmaps/training_config.json']
  predictor: TopDownPredictor
  sleap_version: 1.4.1
  platform: Linux-6.1.123+-x86_64-with-glibc2.35
  command: /usr/local/bin/sleap-track dat

If you're using Chrome you can download your trained models like so:

In [ ]:
# Zip up the models directory
!zip -r trained_models.zip models/

# Download.
from google.colab import files
files.download("/content/trained_models.zip")

  adding: models/ (stored 0%)
  adding: models/courtship.topdown_confmaps/ (stored 0%)
  adding: models/courtship.topdown_confmaps/initial_config.json (deflated 73%)
  adding: models/courtship.topdown_confmaps/labels_pr.train.slp (deflated 68%)
  adding: models/courtship.topdown_confmaps/metrics.val.npz (deflated 0%)
  adding: models/courtship.topdown_confmaps/labels_pr.val.slp (deflated 74%)
  adding: models/courtship.topdown_confmaps/training_config.json (deflated 88%)
  adding: models/courtship.topdown_confmaps/labels_gt.val.slp (deflated 72%)
  adding: models/courtship.topdown_confmaps/metrics.train.npz (deflated 0%)
  adding: models/courtship.topdown_confmaps/training_log.csv (deflated 56%)
  adding: models/courtship.topdown_confmaps/labels_gt.train.slp (deflated 61%)
  adding: models/courtship.topdown_confmaps/best_model.h5 (deflated 8%)
  adding: models/courtship.centroid/ (stored 0%)
  adding: models/courtship.centroid/initial_config.json (deflated 74%)
  adding: models/courtsh

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

And you can likewise download your predictions:

In [ ]:
from google.colab import files
files.download('dataset/drosophila-melanogaster-courtship/20190128_113421.predictions.slp')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In some other browsers (Safari) you might get an error and you can instead download using the "Files" tab in the side panel (it has a folder icon). Select "Show table of contents" in the "View" menu if you don't see the side panel.